# Model sec2sec z użyciem wyrazów

## Stałe

In [11]:
batch_size = 64  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 5000 # Number of samples to train on.
data_path = 'data/pol-eng-short.txt'
clean_data_path = 'data/clean-pol-eng-short.txt'

## Czyszczenie tekstu

In [12]:
import string

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read()
    table = str.maketrans('', '', string.punctuation)
    clean_lines = lines.lower().translate(table)

with open(clean_data_path, 'w', encoding='utf-8') as f:
    f.write(clean_lines)

## Wektoryzacja

In [13]:
# Vectorize the data.
input_texts = []
target_texts = []
input_words = set()
target_words = set()

with open(clean_data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[: num_samples]:
    input_text, target_text = line.split('\t')
    #input_text = '<s> ' + input_text + ' <e>'
    target_text = '\t ' + target_text + ' \n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for word in input_text.split(' '):
        if word not in input_words:
            input_words.add(word)
    for word in target_text.split(' '):
        if word not in target_words:
            target_words.add(word)

In [14]:
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt.split(' ')) for txt in input_texts])
max_decoder_seq_length = max([len(txt.split(' ')) for txt in target_texts])
#max_encoder_seq_length = max([len(txt) for txt in input_texts])
#max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [15]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 5000
Number of unique input tokens: 2046
Number of unique output tokens: 3566
Max sequence length for inputs: 5
Max sequence length for outputs: 11


## Generacja danych

In [16]:
import numpy as np

# slowniki slow
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

# wypelnienie wejsc zerami
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for text_index, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for word_index, (word) in enumerate(input_text.split(' ')):
        encoder_input_data[text_index, word_index, input_token_index[word]] = 1.
    for word_index, (word) in enumerate(target_text.split(' ')):
        decoder_input_data[text_index, word_index, target_token_index[word]] = 1.
        if word_index > 0:
            decoder_target_data[text_index, word_index-1, target_token_index[word]] = 1.

## Model treningowy

In [18]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
#model.save('s2s.h5')

Using TensorFlow backend.


Train on 4000 samples, validate on 1000 samples
Epoch 1/20
4000/4000 [==============================] - 38s 9ms/step - loss: 1.8264 - val_loss: 2.0455
Epoch 2/20
4000/4000 [==============================] - 35s 9ms/step - loss: 1.5922 - val_loss: 2.0688
Epoch 3/20
4000/4000 [==============================] - 48s 12ms/step - loss: 1.5272 - val_loss: 2.0556
Epoch 4/20
4000/4000 [==============================] - 38s 9ms/step - loss: 1.4785 - val_loss: 2.0587
Epoch 5/20
4000/4000 [==============================] - 33s 8ms/step - loss: 1.4406 - val_loss: 2.0447
Epoch 6/20
4000/4000 [==============================] - 34s 8ms/step - loss: 1.4026 - val_loss: 2.0941
Epoch 7/20
4000/4000 [==============================] - 61s 15ms/step - loss: 1.3608 - val_loss: 2.0420
Epoch 8/20
4000/4000 [==============================] - 43s 11ms/step - loss: 1.3051 - val_loss: 1.9946
Epoch 9/20
4000/4000 [==============================] - 36s 9ms/step - loss: 1.2546 - val_loss: 1.9646
Epoch 10/20
4000/4000 

## Model produkcyjny

In [20]:
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense

#model = load_model('s2s-word.h5')
#
#for layer in model.layers:
#    print(layer.get_config())
#    print('\n')
    

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
#reverse_input_char_index = dict(
#    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
   (i, char) for char, i in target_token_index.items())
reverse_input_word_index = dict(
    (i, word) for word, i in input_token_index.items())
reverse_target_word_index = dict(
    (i, word) for word, i in target_token_index.items())



def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: hi
Decoded sentence: wygrałem

-
Input sentence: run
Decoded sentence: 

-
Input sentence: run
Decoded sentence: 

-
Input sentence: run
Decoded sentence: 

-
Input sentence: who
Decoded sentence: ktokto

-
Input sentence: help
Decoded sentence: weź

-
Input sentence: jump
Decoded sentence: 

-
Input sentence: stop
Decoded sentence: przestańsię

-
Input sentence: stop
Decoded sentence: przestańsię

-
Input sentence: hello
Decoded sentence: 

-
Input sentence: hurry
Decoded sentence: onisię

-
Input sentence: i see
Decoded sentence: widzędodomu

-
Input sentence: i see
Decoded sentence: widzędodomu

-
Input sentence: i try
Decoded sentence: uczęsię

-
Input sentence: i won
Decoded sentence: wygrałem

-
Input sentence: oh no
Decoded sentence: nienie

-
Input sentence: relax
Decoded sentence: 

-
Input sentence: smile
Decoded sentence: 

-
Input sentence: attack
Decoded sentence: 

-
Input sentence: cheers
Decoded sentence: zacznij

-
Input sentence: cheers
Decoded sente